# What this document is
This document implements a basic Naive Bayes model to classify spam sms or similar spam email. This article will not go deep into the math and mostly is an implementation guide.

# requirements

1. Python programming language
2. Pandas library
3. Bayes's theorem

# intro
The Naive Bayes model is a machine learning model that is based on the Bayes theorem and probabilities. They are used for solving classification problems, such as determining whether an email is a spam or ham. Bayes theorem can be used to update our beliefs, this property also shows up in the model.

before continuing, let us take a look at Bayes's formula
$\Large{P(A|B)} = \Large\frac{P(B|A).P(A)}{P(B)}$

# how does the model work in theory
First, let me change the variable names in the Bayes formula to better suit our case.

$\Large{P(spam|w)} = \Large\frac{P(w|spam)P(spam)}{P(w)} = \Large\frac{P(w|spam)P(spam)}{P(w|spam)P(spam) + P(w|ham)P(ham)}$

$P(spam)$ denotes the probability of an input to be of class spam and $w$ denotes our feature vector.

## feature vector
Naive Bayes' method handles input as a vector of features. For instance, we may think of an apple as its features like roundness, red, crunchy, freshness, and so forth. Thus, we can define our apple as a vector.

```python
apple = ['round' , 'red' , 'crunchy' , 'fresh']
```

Now that we have the feature vector, we can calculate the probability of the apple being edible if it has these features. 
The initial problem is approachable in the same manner, only here we use the containment of words as the features. Even though we can expand more features to describe the message, I think this satisfies our needs for the moment. Check possible improvements at the end for more information.

## calculating each term

#### Calculating $P(spam)$
This part is rather easy. The term expresses the probability of picking a random message and the message being a scam. Therefore we only need to assure that the data is realistic.

#### Calculating $P(w|spam)$
It turns out that calculating this part is difficult and time-consuming. To solve this challenge Naive Bayes' method assumes that the features are independent which makes our job much easier. We can use joint probability to calculate it

$P(w|spam).P(spam) = P(w,spam) = P(w_{1} , w_{2} , ... , spam) = P(w_{1} | w_{2} , ... , spam).P(w_{2} , w_{3} , ... spam)$

we know that our features are independent or in another way the following expression is true 

$\forall{w_{a},w_{b}\in{w}}:P(w_{a} \cap w_{b}) = P(w_{a}).P(w_{b})$

or

$\forall{w_{a},w_{b}\in{w}}:(w_{a} | w_{b}) = P(w_{a})$

this allows us to simplify our expression to

$P(w|spam).P(spam) = P(w,spam) = P(w_{1}|spam)P(w_{2}|spam)...P(spam) = P(spam).\prod_{i=0}^{n}P(w_{i}|spam)$

#### Calculating $P(w_{i} | spam)$

This expression means the probability of a word being in a message if we already know that its a spam. We can calculate it easily. let $N_{w_{i}|spam}$ be the number of times a word is used in a message and $N_{spam}$ the total number of words in all spam messages, then we can define $P(w_{i} | spam)$ as $\frac{N_{w_{i}|spam}}{N_{spam}}$ this will make much more sense when we get to the implementation

### note

The final formula looks something like this

$\Large{P(spam|w)} = \prod_{i=0}^{n}\left(\frac{P(w_{i}|spam).P(spam)}{P(w_{i}|spam)P(spam) + P(w_{i}|ham)P(ham)}\right) $

Calculating the value of each parenthesis takes four operations, and we multiply n of these therefore, we need 4n operations to calculate the probability. but because we are comparing two probabilities, we can only calculate $P(spam|w)*P(w)$ and $P(ham|w)*P(w)$ and compare these two against each other. It both mathematically makes sense. I tested both methods and found that the accuracy is not affected. Therefore we only need to calculate P(spam,w).

### Special cases

We now know how to calculate each term, though we haven't yet dealt with a special case the case when our word doesn't exist in the dictionary then the probability of $P(w_{i} | spam)$ is zero and this will ruin everything we can solve this problem by using [Laplace smoothing](https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece). Our final formula now looks like the following

$P(w_{i} | spam) = \Large\frac{N_{w_{i}|spam} + a}{N_{spam} + N_{vocab}.a}$

$a$ is a constant, setting it to zero signals that we are not using any smoothing. this may seem unintuitive and certainly felt like that for me but a way you can think about it is when the word is new so what we are left with is $\frac{a}{a.N_{vocab}} $ and setting $a$ to 1 here means that the probability of that word being in a scam message is equal to we picking a random word out of the dictionary.

## Pros and Cons

I talked about the systematic update of beliefs earlier but some other pros include 

1. it allows us to think probabilistically about an object
2. humans can easily understand it
3. high accuracy

some cons are listed below

1. losing the relation between properties 
2. it needs a lot of accurate data with real word probability distribution

# data

I used the [SMS Spam Collection Data set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) from UCI.

## preparing python


In [12]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\amirmahdi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import re

## preparing the data

We can use pandas to read the CSV file into a dataframe


In [16]:
sms_dataset = pd.read_csv('./data/SMSSpamCollection', sep='\t',
header=None, names=['Label', 'SMS'])

## splitting the data

Now we want to split the data into a training set and a testing set 80 percent of the data will be used for training and the rest 20 percent for testing

In [18]:
randomized_data = sms_dataset.sample(frac=1 , random_state=1)
sepration_index = round(len(randomized_data)*0.8)

training_set = sms_dataset[:sepration_index].reset_index(drop=True)
test_set = sms_dataset[sepration_index:].reset_index(drop=True)

In this implementation we will ignore punctutatuation and words case

In [20]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W' , ' ' , regex=True) # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()

## processing the data

In [21]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
   for word in sms:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

We split our messages into tokens. Each token is a word. We will append all the words to a list. `set(vocabulary)` removes all the repeated words. Now we want to now the use count of a word for every message

In [22]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

This creates a dictionary where every word is an entry and it contains a list filled with `len(training_set['SMS'])` many zeros the list represents the number of occurrences in the nth message for example `word_counts_per_sms['free'][0]` is the occurrence count for the word free in the first message. lets now fill the dictionary with data

In [23]:
for index, sms in enumerate(training_set['SMS']):
   for word in sms:
      word_counts_per_sms[word][index] += 1

we can add this dictionary to our data and get a new final dataframe

In [24]:
word_counts = pd.DataFrame(word_counts_per_sms)
training_set_clean = pd.concat([training_set, word_counts], axis=1)
word_counts.head()

,like,oble,evenings,division,neft,nah,lose,arranging,09061744553,bcm,...,08712460324,performance,sending,bollox,thursday,thus,0125698789,seconds,dizzamn,5we
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We want to split the data based on the label to prevent using a filter every time we want to separate our data into spam messages and ham messages

In [25]:
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

now we calculate some of the constant terms like $N_{spam}$

In [26]:
n_spam = spam_messages['SMS'].apply(len).sum()
n_ham = ham_messages['SMS'].apply(len).sum()
n_vocabulary = len(vocabulary)

we can now have everything we need to calculate every $P(w|spam)$ if the following code seems vague go and read the [theory](#how-does-the-model-work-in-theory) again

In [27]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Laplace smoothing
alpha = 1

# Calculate parameters
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham

`n_word_given_spam` as the name suggests, is the same as $N_{w|spam}$ which is the total number of occurrences in all spam messages or the probability of picking a random word out of all spam messages and it is this word. you can reason about other variables similarly. All we now need is a classify function that finishes the job

In [28]:
def classify(message):
   '''
   message: a string
   '''

   # we ignore puncutation and case
   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
   else:
      return 'needs human classification'

we can test the model using the code below

In [29]:
test_set['predicted'] = test_set['SMS'].apply(classify)

correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Label'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1098
Incorrect: 16
Accuracy: 0.9856373429084381


i also tested it with [another dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

# Improvments

1. store the processed data and parameters in a SQL database

2. add new features to properly recognize numbers and URLs. as now numbers are not releated and just fill extra space up.

3. add punctuation handling and